In [1]:
import sys
sys.version

import re
from konlpy.tag import Okt          
from konlpy.tag import Komoran
from collections import Counter
import pandas as pd 

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from wordcloud import WordCloud

from hanspell import spell_checker

### 형태소 분석 및 품사 태깅 라이브러리
* https://konlpy.org/ko/v0.4.3/morph/#

## 한글 폰트 설정

In [2]:
font_path ="C:/Windows/fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=font_path).get_name()
matplotlib.rc('font',family=font_name)

In [69]:
train = pd.read_csv('_data/train.csv', index_col=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  5000 non-null   object
 1   label     5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


In [70]:
test = pd.read_csv('_data/test.csv', index_col=0)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 5000
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  5000 non-null   object
dtypes: object(1)
memory usage: 78.1+ KB


# 형태소 및 단어 분리
* 영어만으로 작성된 리뷰도 있기 때문에 문자열을 제외한 나머지를 공백으로 치환 (re.sub(r'[^\w]+', ' ', x)

In [71]:
okt = Okt()
def preprocessing(df):
    df['document'] = df.document.apply(lambda x : re.sub(r'[^0-9a-zA-Zㅏ-ㅣ가-힣]+', ' ', x).strip())
    df['spell_ck'] = df.document.apply(lambda x : spell_checker.check(x).checked)
    df['tokenized_stem'] = df.document.apply(lambda x : ' '.join(okt.morphs(x, stem=True)))
    df['tokenized_stem2'] = df.spell_ck.apply(lambda x : ' '.join(okt.morphs(x, stem=True)))

    return df

In [72]:
train = preprocessing(train)
train.head()

,document,label,spell_ck,tokenized_stem,tokenized_stem2
id,,,,,
1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0,영상이나 음악이 이쁘다 해도 미화시킨 불륜일 뿐,영상 이나 음악 이 이쁘다 해도 미화 시키다 불륜 일 뿐,영상 이나 음악 이 이쁘다 해도 미화 시키다 불륜 일 뿐
2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯,1,히치콕이 이 영화를 봤다면 분명 손뼉을 쳤을 듯,히치콕 이 이 영화 를 보다 분명 박수 를 치다,히치콕 이 이 영화 를 보다 분명 손뼉 을 치다 듯
3,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,1,괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요,괜찮다 음악 영화 가 또 나오다 따뜻하다 겨울 이 되다 것 같다,괜찮다 음악 영화 가 또 나오다 따뜻하다 겨울 이 되다 것 같다
4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0,아무래도 20년도 지난 작품이라 지금 보기는 너무 유치하다,아무래도 20년 도지난작품이라 지금 보기 는 너무 유치하다,아무래도 20년 도 지난 작품 이라 지금 보기 는 너무 유치하다
5,지금까지의 영화들이 그랬듯 이 영화역시 일본에 대한 미화는 여전하다,0,지금까지의 영화들이 그랬듯 이 영화 역시 일본에 대한 미화는 여전하다,지금 까지의 영화 들 이 그렇다 이 영화 역시 일본 에 대한 미화 는 여전하다,지금 까지의 영화 들 이 그렇다 이 영화 역시 일본 에 대한 미화 는 여전하다


In [73]:
train.tail()

,document,label,spell_ck,tokenized_stem,tokenized_stem2
id,,,,,
4996,좋은 배우들로 3류영화를 찍은 안타까운 영화,0,좋은 배우들로 3류 영화를 찍은 안타까운 영화,좋다 배우 들 로 3 류 영화 를 찍다 안타깝다 영화,좋다 배우 들 로 3 류 영화 를 찍다 안타깝다 영화
4997,진짜 드럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼게 아닌영화,0,진짜 더럽게 재미없다 에드워드 호퍼 그림에 배경 빼고는 볼 게 아닌 영화,진짜 드럽다 재미없다 에드워드 호퍼 그림 에 배경 빼다 볼 게 아니다 영화,진짜 더럽다 재미없다 에드워드 호퍼 그림 에 배경 빼다 볼 게 아니다 영화
4998,가장 실망스러운 영화 지금까지 본영화중,0,가장 실망스러운 영화 지금까지 본 영화 중,가장 실망 스러운 영화 지금 까지 본 영화 중,가장 실망 스러운 영화 지금 까지 보다 영화 중
4999,이런 평점 테러 네이버에서 좀 막아야 하는 것 아닌가,1,이런 평점 테러 네이버에서 좀 막아야 하는 것 아닌가,이렇다 평점 테러 네이버 에서 좀 막다 하다 것 아니다,이렇다 평점 테러 네이버 에서 좀 막다 하다 것 아니다
5000,여주인공이 전작에서는 귀여웠는데 여기선 완전 망했네 실망이다,0,여주인공이 전작에서는 귀여웠는데 여기선 정말 망했네 실망이다,여 주인공 이 전작 에서는 귀엽다 여 기선 완전 망하다 실망 이다,여 주인공 이 전작 에서는 귀엽다 여 기선 정말 망하다 실망 이다


In [92]:
X_train = train['tokenized_stem'].values
y_train = train['label'].values

In [93]:
test = preprocessing(test)
X_test = test['tokenized_stem'].values

In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(1,2),
                       min_df=3,
                       max_df=0.9,
                       tokenizer=okt.morphs,
                       token_pattern=None)
tfidf.fit(X_train)
X_train_okt = tfidf.transform(X_train)
X_test_okt = tfidf.transform(X_test)

In [77]:
SEED = 32

In [95]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

sgd = SGDClassifier(loss='log', random_state=SEED)
param_dist = {'alpha': loguniform(0.0001, 100.0)}
rsv_okt = RandomizedSearchCV(estimator=sgd, param_distributions=param_dist,
                            n_iter=50, random_state=SEED, verbose=1)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=32),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001819D8E9088>},
                   random_state=32, verbose=1)

In [96]:
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

0.8664
{'alpha': 0.00014355463635694167}


In [97]:
submission = pd.read_csv('./_data/sample_submission.csv')
submission

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0
...,...,...
4995,4996,0
4996,4997,0
4997,4998,0
4998,4999,0


# 처리가 필요한 것
1. 맞춤법, 띄어쓰기가 제대로 안되어 있는 경우 --> py-hanspell를 이용하여 맞춤법 교정 후 예측 
2. 자음, 모음만 있는 리뷰 (ex.ㅠㅠ ㅋㅋㅋ) --> 테스트에만 있는 케이스
3. 영어로만 이루어진 리뷰 
4. 숫자 (ex.3류) --> 포함하기로함

In [53]:
inz = [0,0]
inb = [0,0]

for i, s in enumerate(train.document):
    index = train.loc[i+1,'label']
    if ('ㅋ' in s):
        print('[긍정]' if train.loc[i+1, 'label']==1 else '[부정]', i+1, s)
        inz[index]+=1
    elif 'ㅠ' in s:
        print('[긍정]' if train.loc[i+1, 'label']==1 else '[부정]', i+1, s)
        inb[index]+=1
print(f'"ㅋ"이 포함된 부정리뷰 수: {inz[0]}')
print(f'"ㅋ"이 포함된 긍정리뷰 수: {inz[1]}')
print(f'"ㅠ"가 포함된 부정리뷰 수: {inb[0]}')
print(f'"ㅠ"가 포함된 긍정리뷰 수: {inb[1]}')

[긍정] 38 ㅋㅋㅋ역시 피터잭슨 ㅋㅋㅋ완전재밌어 ㅋㅋ
[긍정] 39 영화평점 작성하신분들 왜케 웃김ㅋㅋㅋ보다가 빵터졌어요ㅋㅋ
[부정] 46 촌스럽고 지루해여 근데 유동근아내가 선생님팰땐 웃겨ㅛ음ㅋㅋㅋㅋㅋㅋㅋㅋㅋ
[부정] 49 아나 댓글이 더웃겨 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ
[긍정] 51 재밌게 잘 봤어요ㅋㅋㅋ 남자너무멋져 
[부정] 83 ㅠㅠ 보면 돈이 아까워서 안됩니다 
[부정] 84 좋은 영화인데 재미가 너무 없다 ㅠㅠ
[부정] 94 이유없음과정없음설득력없음BGM도없ㅋ음ㅋ감독맘대로 만든것같은 불친절한영화
[긍정] 110 너무재밌음 ㅋㅋㅋ우리집에 비디오로있는데
[긍정] 117 개 굿 입니다 보세요 ㅋㅋㅋㅋ
[긍정] 146 시원하게 웃을 수 있게 해주는 드라마 ㅋ
[긍정] 163 스티치앓이 중 ㅜㅜ 너무 귀여워서 미치겠다ㅋㅋ 그리고 영화가 완전 감동ㅠㅠ
[긍정] 164 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ좋네요 뭔가통쾌하고 유머러스하네요
[부정] 166 이렇게 원작을 망치기도 힘들겠다ㅋ
[부정] 212 처음에는 나름 좋았는데 아쉽다 허나 배우들이 많이 나오네 ㅋ
[부정] 223 이게 뭔 액션이냐 개차반이네 ㅋㅋㅋㅋㅋㅋ
[긍정] 224 나능좋은데 ㅋ 번봐도 초반전쟁씬 전반내용을 이해못하겠엌 영화는 좋음
[긍정] 239 주군의태양 때문에 보게 됐는데 완전 감동ㅠ ㅠ잊지못할 애니메이션ㅎㅎ
[긍정] 243 재미도 있고 작품성도 있고 ㅋ
[긍정] 261 ㅋㅋㅋㅋㅋ은근히재미엇는데 ㅋㅋㅋㅋㅋㅋㅋㅋㅋ나두초딩때봣어 
[긍정] 264 진쨔 재밌어 요ㅠㅠㅠ 정말정말
[긍정] 275 중간에웃기고 잼게봤음 하얀말 귀여움ㅋㅋ
[부정] 278 코끼리 여자는 왜나오고엔딩은 뭐지 진짜 엔딩은 뭔지아오 ㅋ 빡ㅋ쳐ㅋ
[긍정] 308 재밌당 진짜 ㅋㅋ 시청률 낮은게 이해가 안되용
[긍정] 321 트페 나와서줌 ㅋㅋ 아 갬빗말이에요 갬빗
[긍정] 351 곽지민 매력터지고 얼굴 너무 이쁘다 내스탈 ㅠㅠ 곽지민짱
[긍정] 358 평점이 너무 낮네 ㅋㅋㅋ쨋든 잼잇엇음 ㅋㅋㅋ
[부정] 366 베드

In [31]:
cnt_z =0
cnt_b = 0
for s in test.document:
    if 'ㅋ' in s:
        cnt_z +=1
    elif 'ㅠ' in s:
        cnt_b +=1
        
print(f'"ㅋ"이 포함된 리뷰 수: {cnt_z}')
print(f'"ㅠ"가 포함된 리뷰 수: {cnt_b}')

"ㅋ"이 포함된 리뷰 수: 302
"ㅠ"가 포함된 리뷰 수: 105


# 맞춤법 교정 라이브러리
## Py-Hanspell
* 설치방법
```
git clone https://github.com/ssut/py-hanspell
cd py-hanspell
python setup.py install
```

## PyKoSpacing
* 띄어쓰기 라이브러리 

In [54]:
train.document[3142]

'이거ㅋㅋ옜날에찐짜많이봤는데ㅋㅋㅋㅋㅋㅋㅋ'

In [57]:
spck = spell_checker.check(train.document[3142])
spck.checked

'이거ㅋㅋ옜날에찐짜많이봤는데ㅋㅋㅋㅋㅋㅋㅋ'

In [56]:
temp = '이거 옜날에찐짜많이봤는데'

In [63]:
spell_checker.check(temp).checked

'이거 옛날에 진짜 많이 봤는데'

In [58]:
spck = spell_checker.check(temp)
spck.checked

'이거 옛날에 진짜 많이 봤는데'

* 자음이나 모음이 섞여있을 경우 맞춤법 검사가 제대로 동작하지 않음.
* 자음과 모음만으로 이루어진 리뷰는 없고, 테스트셋에 있더라도 훈련에 없기 때문에 판단하는 것은 어렵다고 생각하여 제거하기로 결정

In [ ]:
okt = Okt()
def preprocessing(df):
    df['document'] = df.document.apply(lambda x : re.sub(r'[^0-9a-zA-Zㅏ-ㅣ가-힣]+', ' ', x).strip())
    df['spell_ck'] = df.document.apply(lambda x : spell_checker.check(x).checked)
    df['tokenized_stem'] = df.document.apply(lambda x : ' '.join(okt.morphs(x, stem=True)))
    df['tokenized_stem2'] = df.spell_ck.apply(lambda x : ' '.join(okt.morphs(x, stem=True)))

    return df

In [101]:
X_train = train['tokenized_stem2'].values
y_train = train['label'].values
X_test = test['tokenized_stem2'].values

tfidf = TfidfVectorizer(ngram_range=(1,2),
                       min_df=3,
                       max_df=0.9,
                       tokenizer=okt.morphs,
                       token_pattern=None)
tfidf.fit(X_train)
X_train_okt = tfidf.transform(X_train)
X_test_okt = tfidf.transform(X_test)

sgd = SGDClassifier(loss='log', random_state=SEED)
param_dist = {'alpha': loguniform(0.0001, 100.0)}
rsv_okt = RandomizedSearchCV(estimator=sgd, param_distributions=param_dist,
                            n_iter=50, random_state=SEED, verbose=1)
rsv_okt.fit(X_train_okt, y_train)

print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
0.8713999999999998
{'alpha': 0.00014355463635694167}


# 맞춤법 교정 전 
* rsv_okt.best_score_: 0.8664 
* rsv_okt.best_params_: 'alpha': 0.00014355463635694167

# 맞춤법 교정 후
* rsv_okt.best_score_: 0.8713999999999998
* rsv_okt.best_params_: 'alpha': 0.00014355463635694167

In [104]:
pred = rsv_okt.predict(X_test_okt)
pred

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [105]:
submission['label']=pred
submission.to_csv('submission.csv', index=False)

# 영어 리뷰

In [147]:
onlyEng =[]
for i, s in enumerate(train.document):
    if len(re.sub(r'[a-zA-Z ]+', '', s))<1:
        print('[긍정]' if train.loc[i+1, 'label']==1 else '[부정]', i+1, s)
        onlyEng.append(s)
print(f'영어로만 쓰여진 리뷰 수:{len(onlyEng)}')

[긍정] 160 very very very good
[긍정] 660 revenge is sweet
[긍정] 722 It gets better and better as it goes on D
[긍정] 2467 divorce D I V O R C E
[긍정] 2718 Miranda Cosgrove A BEST
[긍정] 3694 It s a strange film isn t it
[긍정] 4127 produced be Christopher Nolan
[긍정] 4148 Thuth is out there
[긍정] 4766 No comparison superb
영어로만 쓰여진 리뷰 수:9


영어로 이루어진 리뷰는 모두 긍정리뷰

In [140]:
onlyEng =[]
for i,s in enumerate(test.document):
    if len(re.sub(r'[a-zA-Z ]+', '', s))<1:
        print(i+1,s)
        onlyEng.append(i)
print(f'영어로만 쓰여진 리뷰 수:{len(onlyEng)}')

3 Childhood fantasy
459 Amazing discovery
663 LIFE DOG GREAT
757 It was very good
1024 You saw what you wanted to see
1314 thank you very good job
1326 It was a nice super duper
1387 Best animation ever
1393 Brilliant This is a must see film
2295 What a great drama
2600 It s a quite good movie
3061 harry potter go
3474 gimme the money baby
3655 This movie is I like
3779 
3858 sihoo your best Come on
3998 a girl on the bridge with big eyes
4002 NO WAR DO PINK
4131 
4708 Q Q o
영어로만 쓰여진 리뷰 수:20


In [141]:
test_ori = pd.read_csv('_data/test.csv', index_col=0)
train_ori = pd.read_csv('_data/train.csv', index_col=0)

In [162]:
for i in onlyEng:
    print(f'리뷰: {test_ori.document[i+1]}, 예측: {pred[i]}')

리뷰: Childhood fantasy, 예측: 0
리뷰: "Amazing discovery!", 예측: 0
리뷰: LIFE + DOG = GREAT, 예측: 0
리뷰: It was very good!!!, 예측: 1
리뷰: You saw what you wanted to see., 예측: 0
리뷰: thank you! very good job ^^, 예측: 1
리뷰: It was a nice super duper!!!!, 예측: 1
리뷰: Best animation ever, 예측: 0
리뷰: Brilliant! This is a must-see film, 예측: 1
리뷰: What a great drama!!!, 예측: 1
리뷰: It's a quite good movie!, 예측: 1
리뷰: harry potter go!, 예측: 0
리뷰: gimme the money baby!, 예측: 0
리뷰: This movie is I like, 예측: 0
리뷰: ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ, 예측: 0
리뷰: sihoo,your best! Come on!, 예측: 0
리뷰: a girl on the bridge with big eyes, 예측: 1
리뷰: NO WAR ! DO PINK !!, 예측: 1
리뷰: ㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎㅎ, 예측: 0
리뷰: (￣∇￣)づQ(^.^Q)＼( ˚ ▽ ˚ ) /(/^o^)/, 예측: 1


In [171]:
reviews=[0,0]
for i, s in enumerate(train_ori.document):
    if '!' in s:
        l = '[긍정]' if train_ori.label[i+1]==1 else "[부정]"
        reviews[train_ori.label[i+1]]+=1
        print(l, s)

[긍정] 괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~
[긍정] 가슴 찡했던, 오래 기억에남을 영화!!
[긍정] 즐겁게 본 성장 영화! 화이팅!
[긍정] 멋진 상상력! 아름다운 영화. 여배우 소이의 재발견!
[부정] 개빡치게하는드라마!!질퍽거려너 토나올지경이다!!
[긍정] 닥터후 시리즈는 뭐다??? 관심이다!!
[긍정] 눈물흘리면서 봤습니다. 그리스도인들에게 추천합니다!
[긍정] 재밌게 잘 봤어요ㅋㅋㅋ 남자너무멋져!!
[긍정] 재미있다고!!!!! 블루레이 짱짱 화려한 배경 짱짱
[긍정] 안토니오 반데라스가 어떤 배우인지 보여준다!
[긍정] 내가 좋아하는 배우다 여운이 긴 영화 감동감동!!
[긍정] 이게 예술이야~!! 멋져~!! 모처럼 진짜 멋진 영화를 접했어~!
[긍정] 너무 기분좋게봣습니다~ 적극 추천!!
[부정] 소소에 지루한 이별여행...관심없다!
[긍정] 이 영화는 8점대를 넘어야 정상인 영화다!
[긍정] 진짜 드러나기 싫어하는 호프만의 모습이 인상적!
[긍정] 시원하게 웃을 수 있게 해주는 드라마!ㅋ
[부정] 아직도안끝났나?전파낭비다 차라리다큐를해라 피디도능력없음바꿔라!
[긍정] 외로운 사람들 인연이 더욱 따뜻하게 다가왔던..!!
[긍정] ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ좋네요!뭔가통쾌하고 유머러스하네요
[부정] 시간이 아깝다. (10자 이상!)
[부정] 영화 예고편이 제일 무서워요!
[부정] 이게무슨짓이야!! 이게 이영화 평입니다.
[부정] 진짜 돈 아까워!!열라 내가 해도 그거보다 잘한다-_-
[부정] 정말 배우들이 아깝다...!!
[부정] 그만 좀 우려먹어라!! 재미와 흥미 없으며 이젠 지겹다
[긍정] 오래된 영화지만 이렇게 실감나다니....! 참으로 명작이로고...
[긍정] 보리울의 여름 O.S.T.노래 정말 좋아요!!^.^
[긍정] 정말 잘 만든 영화네요감동!!!영화제작에 참여하신 모든 분들 감사드립니다^^
[긍정] 가슴찡한 우정담에 완전! 감동입니다!
[긍정] 완전 재밌어요~~ 연기도 다들 잘하고 이쁘고 짱짱!!!대박

In [172]:
reviews

[115, 357]

In [178]:
reviews=[0,0]
for i, s in enumerate(train_ori.document):
    if '~' in s:
        l = '[긍정]' if train_ori.label[i+1]==1 else "[부정]"
        reviews[train_ori.label[i+1]]+=1
        print(l, s)
reviews

[긍정] 괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~
[부정] 이런 영화도 만들수가 있구나~
[긍정] 기분전환에 좋은 영화였습니다.~
[긍정] 이게 예술이야~!! 멋져~!! 모처럼 진짜 멋진 영화를 접했어~!
[긍정] 너무 기분좋게봣습니다~ 적극 추천!!
[긍정] 그냥 가슴이 뭉클하고 다시 중고등학교때 추억과 마치 그때느낌이 든다~
[부정] 처음에는 나름 좋았는데 아쉽다...허나 배우들이 많이 나오네~ㅋ
[긍정] 완전 재밌어요~~ 연기도 다들 잘하고 이쁘고 짱짱!!!대박나길♥?♥?♥?
[긍정] 소소한 감동~ 보고 나서 더 생각나는 영화
[긍정] 내가 본 일드 중 쵝오~!!!!!>0<
[긍정] 절로 사람 힘 빠지는 곳 이야기 그래서 더 감동적~
[긍정] 너무 재미있어요. 해피앤딩으로 끝났으면 좋겠어요~태희자은 행복하게 해주세요!
[긍정] ㅋㅋㅋㅋㅋ은근히재미엇는데...ㅋㅋㅋㅋㅋㅋㅋㅋㅋ나두초딩때봣어~~~~~~~~~~
[긍정] 진띰 완전 잘생겼어~!!>< 피터팬,,ㅎ
[긍정] 정말 잘만든 드라마였습니다 . 좋은결말이었지만 너무일찍끝나 아쉽네요~
[긍정] 정우성이라는 배우를 다시 보게된 영화. 허진호 감독의 영화는 다 좋다~
[부정] 공포를 조금 즐기는 편이나 이건,, 도대체 ... 휴~~
[긍정] 윤은혜..주지훈..너무조아~^^
[긍정] 평점 왜이렇게낮나요? 너무사랑스러운 로맨스영화.. 재미있어요 ~~~
[긍정] 잼있다.특이해.아니 별 1나준사람들 이해하기가 쉽지않아요~~~~
[부정] 모냐..~~~어이없는 불륜 영화네
[긍정] 따뜻한 영화~ 너무 좋았어요~~
[부정] 견자단과 성룡의 대결씬과 이쁜 여자들만 볼만~ 나머진 기대 하지 말것
[긍정] 한국판 러브스토리의 최고봉~!
[긍정] 소재를 잘 살려서 재밋게 봄~ 나카시마 미카 짱
[긍정] 최고... 내가 완전 좋아했었던 드라마 ... 또 보고싶당 굿~
[긍정] 갈수록 빠져드네요^^주말이 기달려집니다~
[부정] 롱~롱~롱~테이크!! 환장할 만큼 지루한 영화!!
[부정] tv 전기세가 아깝다 캬악 

[96, 206]

In [179]:
reviews=[0,0]
for i, s in enumerate(train_ori.document):
    if ';' in s:
        l = '[긍정]' if train_ori.label[i+1]==1 else "[부정]"
        reviews[train_ori.label[i+1]]+=1
        print(l, s)
reviews

[부정] 섹시하지도 귀엽지도 않은데 뭘 어쩌라는건지..ㅡㅡ;;
[부정] 조잡한 화면에..우선. 졸작...;;
[부정] 진심 연예인 성우 좀 쓰지마라;; 저게 뭐냐 저게;;;
[부정] 어쩌다 이걸보게되선 ㅡㅡ;; 시간이 진짜 아깝다
[긍정] 모르겠네요. 쟁쟁한 다른 작품들이 있어서 묻힐것 같습니다;;
[긍정] 평점이 너무 낮네; ㅋㅋㅋ쨋든 잼잇엇음 ㅋㅋㅋ
[부정] 우웩;; 이해하려고 노력해도 못하겟음;;
[부정] 영화라기보다는 그냥 투어 촬영 편집본이죠;;; 졸렸습니다.
[긍정] 조연들의 인간다운 연기가 돋보였던 영화 ^^;
[부정] 유치 뽕짝;; OO.. 예전 향수 떠올리는 선플때문에 보다가 토나올뻔했음;;
[부정] -,-;;;; 무간도 아류작인가? 보다 말았음.
[긍정] 1992년도 스릴러가 이정도의 완성도를 자랑하다니;; 수작
[부정] 최악.............;; 정말 별로 중에 별로
[긍정] 액션 정말 잘만들었더라. 사토상 멋있다;ㅋㅋㅋㅋ 즐기기에 충분한 영화
[부정] 이러면안되는거아닌가;; 내 두시간 돌려주세요!!
[부정] 밑에 평가하신 분들 다 스텝들인가요;;
[부정] ..글쎄다...쇼킹하긴하지만 노골적으로 할필요까진..;; 엽기적이다.
[부정] 무슨 이런 영화가.. -_-;; 내가 이해력이 부족한 건가??
[부정] 원작의 묘미는 어디가고 각색 연출 너무 허접해;;
[긍정] 열린입이라고 남드라마헐뜯고하는거아니야^^;이해못하는걸 왜남탓을하지?이해안되네
[부정] 첨엔 좀 볼만한가 했더니;; 시간 아깝네
[부정] 카지노로얄 만든 감독이 만들었는데, 왜 이 모양? ㅡㅡ;
[부정] 어제 케이블에서 하는거 봤는데 다 짤라서 엉망이었던 -_-;
[부정] 나는 관대하다..그것도 아주 ㅡㅡ;;
[부정] 아니왜 코미디영화를 재미없게만들지;;;
[부정] 개후진영화 ㅡㅡ.. 2시간30분동안 지루해죽는줄 ㅡㅡ;;
[부정] 켄신 빠돌이로서 한마디 돈아깝다 -_-;; 제관점은 추억편 영향이 큰듯.
[부정] 맞아요! 지루해요.. 전 전형적인 한국인의 정서라서,,^^;;;
[부

[98, 20]

* !, ~, ; 과 같은 특수문자는 긍/부정을 판단하는데 도움이 될 것이라고 추정